# Load everything

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import findspark  # Get rid of this in DataBricks
findspark.init()  # Get rid of this in DataBricks
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer


In [2]:
from tqdm import tqdm

In [3]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.functions import vector_to_array


In [4]:
config = SparkConf().setAll([('spark.executor.memory', '30g'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "2");
config.set("spark.dynamicAllocation.maxExecutors", "2");
config.set("spark.dynamicAllocation.initialExecutors", "2"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())  # print all the configuration

[('spark.dynamicAllocation.initialExecutors', '2'), ('spark.executor.memory', '30g'), ('spark.executor.id', 'driver'), ('spark.dynamicAllocation.minExecutors', '2'), ('spark.driver.host', '172.18.40.157'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.app.id', 'local-1607148872400'), ('spark.dynamicAllocation.maxExecutors', '2'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.port', '44493'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'proj')]


In [5]:
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system

In [6]:
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')

In [7]:
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  # Jjoin them together

# Drop vechile

In [8]:
df = df.filter(df.cp_type == 'trt_cp')
df = df.drop('cp_type')

# OneHot

In [9]:
indexer = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_cat")
df1 = indexer.fit(df).transform(df)
indexer = StringIndexer(inputCol="cp_time", outputCol="cp_time_cat")
df1 = indexer.fit(df1).transform(df1)
df1 = df1.drop('cp_dose')
df1 = df1.drop('cp_time')

encoder = OneHotEncoder(inputCols=["cp_time_cat", "cp_dose_cat"],
                        outputCols=["cp_time_onehot", "cp_dose_onehot"])

model = encoder.fit(df1)
df1 = model.transform(df1)
df1 = df1.withColumn("cp_time_cols", vector_to_array("cp_time_onehot")).select(df1.columns + [col("cp_time_cols")[i] for i in range(2)])
df1 = df1.withColumn("cp_dose_cols", vector_to_array("cp_dose_onehot")).select(df1.columns + [col("cp_dose_cols")[i] for i in range(1)])
df1 = df1.drop('cp_dose_cat',
 'cp_time_cat',
 'cp_time_onehot',
 'cp_dose_onehot',
)


# Feature Engineering  (credit to Jeff)

In [10]:
gene_feature_names = [name for name in df1.columns if 'g-' in name]
cell_feature_names =  [name for name in df1.columns if 'c-' in name]

df2 = df1.withColumn("gene_max", F.greatest(*gene_feature_names))
df2 = df2.withColumn("gene_min", F.least(*gene_feature_names))
df2 = df2.withColumn("cell_max", F.greatest(*cell_feature_names))
df2 = df2.withColumn("cell_min", F.least(*cell_feature_names))


# Drop high correlation features

In [11]:
feature_columns = gene_feature_names + cell_feature_names  # This came from the previous section
vectorAssembler = VectorAssembler(inputCols = feature_columns, outputCol = 'feats' )
feature_vector = vectorAssembler.transform(df2).select("feats")
# pyspark implementation of determining the correlations
corr_matrix = Correlation.corr(feature_vector, "feats").head()[0]

# Convert the correlation desne matrix and apply mask and to get the indicies where high correlations are observed
# In here, I convert the correlation matrix to numpy, and then use numpy's mask to obtain the lower traingle of the
# matrix. I used numpy becasue pyspark does not have mask.

corr_Array = corr_matrix.toArray()
masked_corr = np.ma.masked_where(np.triu(np.ones_like(corr_Array, dtype=bool)), corr_Array, copy=True)  
idx_high_corr_feats = set(np.argwhere(masked_corr > 0.90)[:,0])  # Set threshold to 90%

# Identify the column to drop and then drop it.
features_to_drop = np.array(feature_columns)[list(idx_high_corr_feats)].tolist()


# Finally

df3 = df2.drop(*features_to_drop)

# Feature trans

???

# Training

In [12]:
final_feature_names = list(set(df3.columns) - set(df_label.columns))

vectorAssembler = VectorAssembler(inputCols = final_feature_names, outputCol = 'feats' )
df4 = vectorAssembler.transform(df3).drop(*final_feature_names)

In [ ]:
df4

In [24]:
def train_individual_label(df, label_name):
    
    
    if df.filter(df[label_name] == 1).count() >= 2:
    
        temp_df = df.select('feats', label_name)

        # stratify split of the dataframe for train-test split
        seed = 42
        fractions = {1: 0.8, 0: 0.8}
        train_df = temp_df.stat.sampleBy(label_name, fractions, seed, )
        test_df =  temp_df.subtract(train_df)


        # Over/down sampling of the training dataframe, becasue of the imbalanced class label

        activation_samples = train_df.filter(train_df[label_name] == 1)
        non_activation_samples = train_df.filter(train_df[label_name] == 0)
        ratio = activation_samples.count() / non_activation_samples.count()

        default_down_sample_ratio = 0.5  # This can be changed, but for now,i am just setting it as 0.5

        upsample_ratio = default_down_sample_ratio / ratio

        activation_samples_up = activation_samples.sample(True, upsample_ratio, 42)   # Upsample the activation samples

        non_activation_samples_down = non_activation_samples.sample(True, default_down_sample_ratio, 42)  # Meanwhile, downsampling the non-activated samples 


        final_train_df = activation_samples_up.union(non_activation_samples_down).orderBy(F.rand())


        # Feature selection using random forrest before using other models

        clf = RandomForestClassifier(numTrees=20, maxDepth=5, labelCol = label_name, featuresCol='feats',  seed=42)


        model = clf.fit(final_train_df)

        feature_importance = model.featureImportances.toArray()
        important_feature_idx = feature_importance.argsort()[-int(0.1 * len(feature_importance)):]  # Only get the top 10%, according to 
                                                                                                    # the feature importance from random forrest classifier



        # Now, after getting the index, filter the feature vector based on the above feature importance index

        slicer = VectorSlicer(inputCol="feats", outputCol="feats_sub", indices=important_feature_idx)
        final_train_df_sub_feats =  slicer.transform(final_train_df).drop('feats')
        final_test_df_sub_feats = slicer.transform(test_df).drop('feats')



        # Finally, use CV to train the model and get the best parameters

        lr = LogisticRegression(maxIter=10, featuresCol='feats_sub',  labelCol = label_name)
        paramGrid = ParamGridBuilder() \
                        .addGrid(lr.regParam, [ 0.1, 0.01]) \
                        .addGrid(lr.elasticNetParam, [1,  0])\
                        .build()

        
        evaluator = MulticlassClassificationEvaluator(labelCol=label_name, metricName='logLoss')
        crossval = CrossValidator(estimator=lr,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=3)  

        cvModel = crossval.fit(final_train_df_sub_feats)
        prediction_df = cvModel.transform(final_test_df_sub_feats)

        cvModel.save(f"./logistics/{label_name}.model")
        prediction_df.write.save(f"./logistics/{label_name}_prediction_df.parquet", format="parquet")
        final_train_df_sub_feats.write.save(f"./logistics/{label_name}_train_df.parquet", format="parquet")

        log_loss = evaluator.evaluate(prediction_df)

        with open(f"./logistics/log.log", 'a') as f:
            f.write(str(log_loss) + '\n') 

        return (True, log_loss)
    else:
        return (False, None)

In [64]:
train_individual_label = None

Stratify 
ref: https://stackoverflow.com/questions/47637760/stratified-sampling-with-pyspark/47672336

In [ ]:
with open(f"./logistics/log.log", 'w') as f:
    pass


temp_dict = {}
for name in tqdm(df_label.columns[1:]):
    temp_dict[name] = train_individual_label(df4,  name)

 24%|██▍       | 49/206 [9:05:15<32:42:14, 749.90s/it]

['5-alpha_reductase_inhibitor',
 '11-beta-hsd1_inhibitor',
 'acat_inhibitor',
 'acetylcholine_receptor_agonist',
 'acetylcholine_receptor_antagonist',
 'acetylcholinesterase_inhibitor',
 'adenosine_receptor_agonist',
 'adenosine_receptor_antagonist',
 'adenylyl_cyclase_activator',
 'adrenergic_receptor_agonist',
 'adrenergic_receptor_antagonist',
 'akt_inhibitor',
 'aldehyde_dehydrogenase_inhibitor',
 'alk_inhibitor',
 'ampk_activator',
 'analgesic',
 'androgen_receptor_agonist',
 'androgen_receptor_antagonist',
 'anesthetic_-_local',
 'angiogenesis_inhibitor',
 'angiotensin_receptor_antagonist',
 'anti-inflammatory',
 'antiarrhythmic',
 'antibiotic',
 'anticonvulsant',
 'antifungal',
 'antihistamine',
 'antimalarial',
 'antioxidant',
 'antiprotozoal',
 'antiviral',
 'apoptosis_stimulant',
 'aromatase_inhibitor',
 'atm_kinase_inhibitor',
 'atp-sensitive_potassium_channel_antagonist',
 'atp_synthase_inhibitor',
 'atpase_inhibitor',
 'atr_kinase_inhibitor',
 'aurora_kinase_inhibitor',
 '

# SCRAP

In [60]:
my_set = {'Geeks', 'for', 'geeks'} 
  
s = list(my_set)

TypeError: 'list' object is not callable

In [62]:
type(list)

list

In [ ]:
cell_feature_names

In [32]:
df.select("cp_type").distinct().show()

+-------+
|cp_type|
+-------+
| trt_cp|
+-------+



In [ ]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [6]:
from pyspark.sql.functions import lit


In [7]:
lit(0.8)

Column<b'0.8'>

In [8]:
from pyspark.sql.functions import lit
list = [(2147481832,23355149,1),(2147481832,973010692,1),(2147481832,2134870842,1),(2147481832,541023347,1),(2147481832,1682206630,1),(2147481832,1138211459,1),(2147481832,852202566,1),(2147481832,201375938,1),(2147481832,486538879,1),(2147481832,919187908,1),(214748183,919187908,1),(214748183,91187908,1)]
df = spark.createDataFrame(list, ["x1","x2","x3"])

In [9]:
df

DataFrame[x1: bigint, x2: bigint, x3: bigint]

In [10]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [12]:
fractions

{2147481832: 0.8, 214748183: 0.8}

In [15]:
df.select("x1").distinct().withColumn("fraction", lit(0.8)).show()

+----------+--------+
|        x1|fraction|
+----------+--------+
|2147481832|     0.8|
| 214748183|     0.8|
+----------+--------+



In [ ]:
temp_df = features_and_targets.withColumn('target_vector', (vector_to_string(array([features_and_targets[col] for col in target_names])))).select(['sig_id', 'target_vector'])
string_indexer = StringIndexer(inputCol = 'target_vector', outputCol = 'target')
string_indexer_model = string_indexer.fit(temp_df)
temp_df = string_indexer_model.transform(temp_df).drop('target_vector')

data = features_and_targets.join(temp_df, features_and_targets.sig_id == temp_df.sig_id, how = 'inner').drop(temp_df.sig_id)